## Automated anisotropic resistivity inversion for efficient formation evaluation and uncertainty quantification

### Misael M. Morales, Michael Pyrcz, Carlos Torres-Verdin, 2024
***

In [ ]:
from main import *
ari = ARI()

In [ ]:
case1, case2 = ari.load_data()

In [ ]:
case1_qinv = ari.quadratic_inversion(case1)
case1_inv  = ari.resistivity_inversion(case1)

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15,12), sharey=True, facecolor='white')
ax1, ax2, ax3, ax4, ax5 = axs

ax11, ax12 = ax1.twiny(), ax1.twiny()
ari.plot_curve(ax1,  case1_inv, 'GR',      lb=0, ub=150, color='g',    units='API',  pad=0)
ari.plot_curve(ax11, case1_inv, 'Csh_lin', lb=0, ub=1,   color='gray', units='frac', pad=8)
ari.plot_curve(ax12, case1_inv, 'Csh',     lb=0, ub=1,   color='k',    units='frac', ls='--', pad=16)

ax21, ax22 = ax2.twiny(), ax2.twiny()
ari.plot_curve(ax2,  case1_inv, 'AT10', lb=0.2, ub=50, color='r', units='$\Omega.m$', semilog=True, pad=0)
ari.plot_curve(ax21, case1_inv, 'AT90', lb=0.2, ub=50, color='b', units='$\Omega.m$', semilog=True, pad=8)
ari.plot_curve(ax22, case1_inv, 'Rs',   lb=0.2, ub=200, color='k', units='$\Omega.m$', alpha=0.75, semilog=True, pad=16)

ax31, ax32 = ax3.twiny(), ax3.twiny()
ari.plot_curve(ax3,  case1_inv, 'Rv',     lb=0.2, ub=100, color='darkred',  units='$\Omega.m$',   semilog=True, pad=0)
ari.plot_curve(ax31, case1_inv, 'Rv_sim', lb=0.2, ub=100, color='k', units='$\Omega.m$', ls='--', alpha=0.75, semilog=True, pad=8)
ari.plot_curve(ax32,  case1_inv, 'Rv_err', lb=1e-9, ub=100, color='red', units='%', alpha=0.5, pad=16)

ax41, ax42 = ax4.twiny(), ax4.twiny()
ari.plot_curve(ax4,  case1_inv, 'Rh',     lb=0.2, ub=100, color='darkblue',  units='$\Omega.m$',  semilog=True, pad=0)
ari.plot_curve(ax41, case1_inv, 'Rh_sim', lb=0.2, ub=100, color='k', units='$\Omega.m$', alpha=0.75, ls='--', semilog=True, pad=8)
ari.plot_curve(ax42, case1_inv, 'Rh_err', lb=1e-9, ub=100, color='blue', units='%', alpha=0.5, pad=16)

ax51, ax52 = ax5.twiny(), ax5.twiny()
ari.plot_curve(ax5,  case1_inv, 'fun',      lb=0, ub=0.5,  color='k', pad=0)
ari.plot_curve(ax51, case1_inv, 'nfev',     lb=50, ub=300, color='g', alpha=0.75, pad=8)
ari.plot_curve(ax52, case1_inv, 'norm_jac', lb=0, ub=25,  color='m', alpha=0.75, pad=16)

plt.gca().invert_yaxis()
plt.show()

In [ ]:
from torch.optim import LBFGS
from torch.linalg import norm as torch_norm

In [ ]:
import torch
import pandas as pd
from torch.optim import LBFGS
from torch.linalg import norm

def objective(variables, Rv, Rh):
    Csh, Rs = variables
    eq1 = (Csh * Rvsh + (1 - Csh) * Rs) - Rv
    eq2 = (Csh / Rhsh + (1 - Csh) / Rs) - (1 / Rh)
    eqs = [eq1, eq2]
    return norm(torch.tensor(eqs)) + lambda_reg * norm(variables)

def inversion(df, method='L-BFGS-B'):
    res_aniso = df[['Rv', 'Rh']]
    sol, fun, jac, nfev = [], [], [], []
    for _, row in res_aniso.iterrows():
        Rv_value, Rh_value = row['Rv'], row['Rh']
        # Initial guess for variables
        initial_variables = torch.tensor([0.5, 1.0], requires_grad=True)
        # Define the optimizer
        optimizer = LBFGS([initial_variables])
        # Define the optimization closure
        def closure():
            optimizer.zero_grad()
            loss = objective(initial_variables, Rv_value, Rh_value)
            loss.backward()
            return loss
        # Optimize the objective function
        optimizer.step(closure)
        # Get the optimized values
        optimized_variables = initial_variables.detach().numpy()
        # Compute function value and Jacobian
        function_value = objective(torch.tensor(optimized_variables), Rv_value, Rh_value).item()
        jac_value = torch.autograd.functional.jacobian(lambda x: objective(x, Rv_value, Rh_value), torch.tensor(optimized_variables)).detach().numpy()
        # Store results
        sol.append({'Rv': Rv_value, 'Rh': Rh_value, 'Csh': optimized_variables[0], 'Rs': optimized_variables[1]})
        fun.append(function_value)
        jac.append(jac_value)
        nfev.append(1)  # Assuming each optimization step counts as one function evaluation
    sol = pd.DataFrame(sol, index=res_aniso.index)
    sol['fun'], sol['nfev'], sol['jac1'], sol['jac2'] = fun, nfev, [j[0] for j in jac], [j[1] for j in jac]
    sol['norm_jac'] = norm(torch.tensor(jac), axis=1)
    return sol


In [ ]:
sol = inversion(case1)
sim = simulate(sol)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10,5), sharex=True, sharey=True, facecolor='white')
ax1, ax2, ax3 = axs

ax11 = ax1.twinx()
ax1.semilogy(sol.index, sol['Rs'], 'k', label='Rs')
ax11.plot(sol.index, sol['Csh'], 'g', label='Csh')
ax1.legend(loc='lower left'); ax11.legend(loc='lower right')

ax2.semilogy(sol.index, sol['Rv'], 'r', label='Rv')
ax2.semilogy(sim.index, sim['Rv_sim'], 'k--', label='Rv_sim')
ax2.legend(loc='lower right')

ax3.semilogy(sol.index, sol['Rh'], 'b', label='Rh')
ax3.semilogy(sim.index, sim['Rh_sim'], 'k--', label='Rh_sim')
ax3.legend(loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
Rvsh = torch.varialbe

***
# END